<a href="https://colab.research.google.com/github/VCucuiat/PlayingOut/blob/master/movement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- identify same child_id from one frame to another - for all the trajectories for all limbs from the previous 5 - 10? frames, identify the most likely canditate from the next frame

- draw movement trajectories
- start / stop movement?
- synchronisation between students' movement? posture? to what extent are the trajectories similar? feature extraction for comparison? time wrapping? scale normalisation?
- amount of physical movement?

Less achievable:
- calculation of limb length (needs depth)
- height calculation (needs limb length)
- proximity to each other (needs depth, possibly aided by height)


In [ ]:
import os

directory = 'drive/My Drive/Colab Notebooks/openpose/D2PaOutPoles3T/555-805/output/';

json_files = [];
for filename in os.listdir(directory):
    if filename.endswith(".json") & ( filename != '0_compare_keypoints.json'):
         json_files.append(filename);

json_files.sort();
print(len(json_files));




3894


In [ ]:
import json

people_per_frame = [];

for x in range(0,len(json_files)):
  people = 0;
  file = directory + json_files[x];

  with open(file, 'r') as json_file_each:
    data_each = json.load(json_file_each)
    for p in data_each['people']:  
      people = people + 1;

  people_per_frame.append(people);

#print(people_per_frame);

In [ ]:
import json

people_per_frame = [];

for x in range(2300,2500):
  file = directory + json_files[x];

  with open(file, 'r') as json_file_each:
    data_each = json.load(json_file_each)
    print(file);
    for p in data_each['people']: 
      print(p['person_id']);

drive/My Drive/Colab Notebooks/openpose/D2PaOutPoles3T/555-805/output/555-805_000000002300_keypoints.json
1
3
2
drive/My Drive/Colab Notebooks/openpose/D2PaOutPoles3T/555-805/output/555-805_000000002301_keypoints.json
1
3
2
drive/My Drive/Colab Notebooks/openpose/D2PaOutPoles3T/555-805/output/555-805_000000002302_keypoints.json
1
3
2
drive/My Drive/Colab Notebooks/openpose/D2PaOutPoles3T/555-805/output/555-805_000000002303_keypoints.json
1
3
2
drive/My Drive/Colab Notebooks/openpose/D2PaOutPoles3T/555-805/output/555-805_000000002304_keypoints.json
1
3
2
drive/My Drive/Colab Notebooks/openpose/D2PaOutPoles3T/555-805/output/555-805_000000002305_keypoints.json
1
2
3
drive/My Drive/Colab Notebooks/openpose/D2PaOutPoles3T/555-805/output/555-805_000000002306_keypoints.json
1
3
2
drive/My Drive/Colab Notebooks/openpose/D2PaOutPoles3T/555-805/output/555-805_000000002307_keypoints.json
1
3
2
drive/My Drive/Colab Notebooks/openpose/D2PaOutPoles3T/555-805/output/555-805_000000002308_keypoints.jso

In [ ]:
import os
def is_non_zero_file(fpath):  
    return os.path.isfile(fpath) and os.path.getsize(fpath) > 0

In [ ]:
import json;

def update_compare_keypoints(update_file):

  # if the file doesn't exit, create it, and read the first character to check if it is empty or not
  compare_file = directory + '0_compare_keypoints.json';
  #outfile = open(compare_file, "w+");
  #outfile.seek(0);
  #one_char = outfile.read(1);
  #print('one_char: ', one_char)

  # load that data in the openpose file
  with open(update_file, 'r+') as infile:
    data_read = json.load(infile);
  
  # if data in the compare file, update existing ids and append new ones from the openpose file
  if is_non_zero_file(compare_file):
    items = [];

    for q in data_read['people']:
      write = True;  

      with open(compare_file, 'r+') as outfile:
        data_write = json.load(outfile);
        for p in data_write['people']:
          if int(q['person_id']) == int(p['person_id']):
            print('update: ', q['person_id']);
            p['pose_keypoints_2d'] = q['pose_keypoints_2d'];
            p['last_update'] = update_file;
            outfile.seek(0);        # <--- resets file position to the beginning.
            json.dump(data_write, outfile, indent=4);
            outfile.truncate();
            write = False;

      if write == True:
        entry = {"person_id": q['person_id'], "last_update": update_file, "pose_keypoints_2d": q['pose_keypoints_2d']}
        items.append(entry);

    if len(items) > 0:
      for i in range(0,len(items)):
        print('append: ', items);
        with open(compare_file, mode='r') as outfile:
          data_write = json.load(outfile);
        with open(compare_file, mode='w') as outfile:
          json.dump([], outfile);
        with open(compare_file, mode='w') as outfile:
          data_write['people'].append(items[i])
          json.dump(data_write, outfile);

  #if no data in the compare file, append all entries from the openpose file
  else:
    print('empty update ');
    items = [];

    for q in data_read['people']:
      entry = {"person_id": q['person_id'], "last_update": update_file, "pose_keypoints_2d": q['pose_keypoints_2d']}
      items.append(entry);
    people = {"people": items};

    with open(compare_file, mode='w+') as outfile: 
      json.dump(people, outfile);
      outfile.truncate(); 

  



In [ ]:
import json;
import numpy as np;
import copy;

def set_people_id(file):
  print(file);

  compare_file = directory + '0_compare_keypoints.json';
  with open(compare_file, 'r+') as json_file1:
    data_1 = json.load(json_file1)

  with open(file, 'r') as json_file2:
    data_2 = json.load(json_file2)

  #ids of the people in the compare file
  compare_people_id = [];
  for p in data_1['people']:
    compare_people_id.append(p['person_id']);
  print(compare_people_id);

  #no of people in the current frame to set
  no_people = 0;
  for q in data_2['people']:
    no_people = no_people + 1;  
  #set no_people to the minimum between the no_people in the current frame, and the no_people we can compare to
  #if more people in the current frame, then we set new ids for the remaining people - not in this function
  #if less people in the current frame, then only set that many
  print('frame people: ', no_people);
  no_people = min(len(compare_people_id),no_people);
  print('set people: ', no_people);

  #################################################################################################################
  
  #calculate the difference between each person in the curent frame, and the latest saved coordinates of each previously identified person in compare_file
  diff_all = [];

  for q in data_2['people']:
    #for each person in the new file not already wih an id, calculate the difference between them and each person from the compare_file
    if int(q['person_id']) == -1:
      diff_each = [];
      
      for p in data_1['people']:

        difference = 0;
        confident_joints = 0; #where both the latest compare coordinates and the curent coordinates are >0, therefore have a value

        for i in range(2, 75):
          if (i+1)%3 == 0:
            #every third value is a confidence score or the joint recognition from openpose
            x1 = p['pose_keypoints_2d'][i-2];
            y1 = p['pose_keypoints_2d'][i-1];
            c1 = p['pose_keypoints_2d'][i];

            x2 = q['pose_keypoints_2d'][i-2];
            y2 = q['pose_keypoints_2d'][i-1];
            c2 = q['pose_keypoints_2d'][i];

            if ((c1 > 0) and (c2 > 0)):
              difference = difference + (max(x1,x2) - min(x1, x2)) + (max(y1,y2) - min(y1,y2));
              confident_joints = confident_joints + 1;
          
        if confident_joints > 0:
          difference = difference/(confident_joints*2);
        else:
          # this means there are no joins where both in compare and current frames the values are >0, so we basically cannot determine which person it is a best match for
          difference = 999;     

        diff_each.append(difference);

      diff_all.append(diff_each);

  print('diff_all: '); print(diff_all);

  #################################################################################################################

  # in order to assign the ids of the best matched people first, create store the lowest values for each person in the current frame in minimum_movement, and set each id in the order from minimum_movement
  # this prevents cases where a person is very close to 2 people, and rather than taking in the order of the closest, we take it in the order in which the people were save in the json file, which is prone to errors
  matched_diff_o = [];
  matched_diff_o = copy.deepcopy(diff_all);

  if len(diff_all) > 0: #if any people in the new file with id -1 (sometime there are none if I re-run it when some of the files are already done, but not all)
    minimum_movement = [];
    # store the closes person from the previous frame for each person in the new frame in minimum_movement
    # so that I don't just set the id of the closest person from the previous frame, but I set the closest people in the new frame first
    # for cases where there are more people in the new frame than there are in the previous frame, or more than 2 people very close to each other
    for a in range(0,len(matched_diff_o)):
      minimum_movement.append(min(matched_diff_o[a]));
      match_id = matched_diff_o[a].index(min(matched_diff_o[a]));
      for b in range(0,len(matched_diff_o)):
        matched_diff_o[b][match_id] = 99999999;      
      print('matched_diff_o: '); print(matched_diff_o);

    print('minimum_movement: '); print(minimum_movement);

    # now save the people_ids I should set from compare_file into people_ids, to then save into the json file
    # only set ids for the no of people in the current frame - could be more, or less than compare_file, where I store all the people that have been encountered in any past frame
    people_ids = [];
    for z in range(0,len(diff_all)):
      people_ids.append(-1);

    for z in range(0,no_people):
      closest_match = minimum_movement.index(min(minimum_movement));
      print('closest_match: '); print(closest_match);
      print('min(diff_all[closest_match]): '); print(min(diff_all[closest_match]));
      print('diff_all[closest_match].index(min(minimum_movement)): '); print(diff_all[closest_match].index(min(minimum_movement)));
      print('compare_people_id[diff_all[closest_match].index(min(diff_all[closest_match]))]: '); print(compare_people_id[diff_all[closest_match].index(min(minimum_movement))]);
      people_ids[closest_match] = compare_people_id[diff_all[closest_match].index(min(minimum_movement))];
      minimum_movement[closest_match] = 99999999; 

    print('people_ids: '); print(people_ids);    
    print('minimum_movement: '); print(minimum_movement);    

    i = 0;
    with open(file, 'r+') as json_file2:
      data_2 = json.load(json_file2);
      
      for q in data_2['people']:   
          if (int(q['person_id']) == -1):
            q['person_id'] = people_ids[i];
            json_file2.seek(0);        # <--- resets file position to the beginning.
            json.dump(data_2, json_file2, indent=4);
            json_file2.truncate();
            i = i+1;

  print('number of compared people: ', len(compare_people_id));
  return(len(compare_people_id));


In [ ]:
import json
import os

def reset_ids(x,y):

  for i in range(x,y):
    file = directory + json_files[i];

    with open(file, 'r+') as json_file_reset:
      data_reset = json.load(json_file_reset)
      for p in data_reset['people']:  
        p['person_id'] = -1;
        json_file_reset.seek(0);        # <--- resets file position to the beginning.
        json.dump(data_reset, json_file_reset, indent=4);
        json_file_reset.truncate();
  
  compare_file = directory + '0_compare_keypoints.json';
  if is_non_zero_file(compare_file):
    os.remove(compare_file);

reset_ids(2560, 3894)

In [ ]:
import json

people = 0;

first_file = directory + json_files[0];
print(first_file);

with open(first_file, 'r+') as json_file_first:
    data_first = json.load(json_file_first);

    for p in data_first['people']:
      print(p['person_id']);
      people = people + 1;
      p['person_id'] = people;
      json_file_first.seek(0);        # <--- resets file position to the beginning.
      json.dump(data_first, json_file_first, indent=4);
      json_file_first.truncate();

    for p in data_first['people']:
      print(p['person_id']);

update_compare_keypoints(first_file);

print('yo!')

drive/My Drive/Colab Notebooks/openpose/D2PaOutPoles3T/555-805/output/555-805_000000000000_keypoints.json
1
2
3
1
2
3
empty update 
yo!


In [ ]:
import json

for x in range(3000,3894):
  file = directory + json_files[x];

  # set the ids comparing to the values in the compare file and return how many people the ids were set
  no_compare_people = set_people_id(file);

  y = people_per_frame[x] - no_compare_people; 
  people_id_extra = people_per_frame[x];
  print('y: ', y);
  
  # if more people in the current frame than in any previous frames (stored on the compare file), set new ids
  if y > 0:
    with open(file, 'r+') as json_file:
      data = json.load(json_file);
      
      for q in data['people']:  
        if (int(q['person_id']) == -1):
          q['person_id'] = people_id_extra;
          json_file.seek(0);        # <--- resets file position to the beginning.
          json.dump(data, json_file, indent=4);
          json_file.truncate();
          people_id_extra = people_id_extra + 1;
  
  update_compare_keypoints(file);


Streaming output truncated to the last 5000 lines.
closest_match: 
0
min(diff_all[closest_match]): 
1.8629565217391235
diff_all[closest_match].index(min(minimum_movement)): 
0
compare_people_id[diff_all[closest_match].index(min(diff_all[closest_match]))]: 
1
people_ids: 
[1, 2, 5]
minimum_movement: 
[99999999, 99999999, 99999999]
number of compared people:  5
y:  -2
update:  1
update:  2
update:  5
drive/My Drive/Colab Notebooks/openpose/D2PaOutPoles3T/555-805/output/555-805_000000003788_keypoints.json
[1, 2, 3, 4, 5]
frame people:  3
set people:  3
diff_all: 
[[1.1727599999999985, 96.06576190476191, 71.81063333333334, 88.91976470588233, 512.2377608695652], [95.57035714285712, 1.6207619047619035, 140.76125, 67.41633333333336, 431.9512368421053], [513.1348695652175, 431.7903421052632, 529.6862666666667, 439.72353124999995, 2.2263695652173916]]
matched_diff_o: 
[[99999999, 96.06576190476191, 71.81063333333334, 88.91976470588233, 512.2377608695652], [99999999, 1.6207619047619035, 140.7612

In [ ]:
x=2000;
while x <= 2520:
  #previous file
  file_test = directory + json_files[x];
  print(file_test);

  with open(file_test, 'r') as json_file_test:
    data_test = json.load(json_file_test)
    
    for p in data_test['people']:
      print(p['person_id']);
      print(p['pose_keypoints_2d'][0], ' ', p['pose_keypoints_2d'][1], ' ', p['pose_keypoints_2d'][2]);
      print(p['pose_keypoints_2d'][3], ' ', p['pose_keypoints_2d'][4], ' ', p['pose_keypoints_2d'][5]);                                                                             
      print(p['pose_keypoints_2d'][69], ' ', p['pose_keypoints_2d'][70], ' ', p['pose_keypoints_2d'][71]);                                                                          
      print(p['pose_keypoints_2d'][72], ' ', p['pose_keypoints_2d'][73], ' ', p['pose_keypoints_2d'][74]);
  x = x + 1;

Streaming output truncated to the last 5000 lines.
1139.52   721.586   0.671393
2
1454.45   309.472   0.849616
1492.6   350.716   0.872286
1516.14   701.044   0.304929
1545.68   715.824   0.802836
3
315.275   489.054   0.767294
244.873   589.101   0.820791
350.732   1074.92   0.0511649
0   0   0
drive/My Drive/Colab Notebooks/openpose/D2PaOutPoles3T/555-805/output/555-805_000000002208_keypoints.json
1
1183.57   309.483   0.868113
1186.64   359.502   0.932254
1116.02   724.594   0.84358
1139.57   718.551   0.760926
2
1445.63   315.421   0.965292
1478.06   362.348   0.862829
1513.35   712.724   0.365747
1542.88   718.701   0.733066
3
321.419   491.981   0.793342
247.838   580.36   0.812751
365.475   1074.93   0.0645354
365.356   1074.9   0.139775
drive/My Drive/Colab Notebooks/openpose/D2PaOutPoles3T/555-805/output/555-805_000000002209_keypoints.json
1
1174.84   312.407   0.866151
1183.7   359.529   0.898663
1116.03   724.576   0.870591
1139.63   718.544   0.770913
2
1430.94   318.495   

In [ ]:
diff = [197.60771428571428, 8.156312499999977]
print(min(diff))

8.156312499999977
